## Setup

In [1]:
%load_ext autoreload
%autoreload 2
%cd ..

/home/nassim/dev/mlops-project


In [2]:
from src.utils import *
from src.models import *
from src.data import *

import wandb
import numpy as np
import pandas as pd
import random
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import pandas as pd
import math
import itertools
from collections import Counter
import os

In [3]:
wandb.login()

wandb: Currently logged in as: dodicin (use `wandb login --relogin` to force relogin)


True

In [4]:
batch_size = 128
num_classes = 10
val_ratio = 0.15
num_workers = 8
dm = FashionMNISTDataModule(batch_size=batch_size, num_classes=num_classes, val_ratio=val_ratio, num_workers=8)
dm.setup()

val_samples = next(iter(dm.val_dataloader()))
val_imgs, val_labels = val_samples[0], val_samples[1]
val_imgs.shape, val_labels.shape

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting ./data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw

/home/nassim/miniconda3/envs/ml-env/lib/python3.9/site-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448222085/work/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


(torch.Size([128, 1, 28, 28]), torch.Size([128]))

In [5]:
n = len(dm.train)+len(dm.val)+len(dm.test)

print("Dataset split:\n"+
      "Train: {:.2f}\n".format(len(dm.train)/n)+
      "Val: {:.2f}\n".format(len(dm.val)/n)+
      "Test: {:.2f}\n".format(len(dm.test)/n)
     )

Dataset split:
Train: 0.73
Val: 0.13
Test: 0.14



In [7]:
# Checking class weights
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
num_classes = len(class_names)

y_train = dm.train.dataset.targets
counts = np.bincount(y_train)
class_weights = {i: 1.0/count for i, count in enumerate(counts)}
print (f"counts: {counts}\nweights: {class_weights}")

counts: [6000 6000 6000 6000 6000 6000 6000 6000 6000 6000]
weights: {0: 0.00016666666666666666, 1: 0.00016666666666666666, 2: 0.00016666666666666666, 3: 0.00016666666666666666, 4: 0.00016666666666666666, 5: 0.00016666666666666666, 6: 0.00016666666666666666, 7: 0.00016666666666666666, 8: 0.00016666666666666666, 9: 0.00016666666666666666}


In [21]:
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks import Callback
from pytorch_lightning.callbacks import ModelCheckpoint

input_dim = val_imgs.shape[1:]

model = ResNet18(input_dim, num_classes)

In [22]:
MODEL_CKPT_PATH = './model/'
MODEL_CKPT = './model/model-{epoch:02d}-{val_loss:.2f}'

checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',
    filename=MODEL_CKPT,
    save_top_k=3,
    mode='min')

early_stop_callback = EarlyStopping(
   monitor='val_loss',
   patience=3,
   verbose=False,
   mode='min'
)

wandb_logger = WandbLogger(project='mlops-project', job_type='train', entity='dodicin')

# Initialize a trainer
trainer = pl.Trainer(max_epochs=2,
                     progress_bar_refresh_rate=20,
                     logger=wandb_logger,
                     callbacks=[early_stop_callback,
                                checkpoint_callback,
                                ImagePredictionLogger(val_samples, 64)],
                     checkpoint_callback=True)

trainer.fit(model, dm)
trainer.test()
wandb.finish()

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs

  | Name              | Type             | Params
-------------------------------------------------------
0 | feature_extractor | ResNet           | 11.7 M
1 | classifier        | Linear           | 10.0 K
2 | loss              | CrossEntropyLoss | 0     
-------------------------------------------------------
10.0 K    Trainable params
11.7 M    Non-trainable params
11.7 M    Total params
46.763    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': 0.7856000065803528, 'test_loss': 0.6007950305938721}
--------------------------------------------------------------------------------


_runtime,338
_timestamp,1630427127
_step,22
train_loss_step,0.52375
train_acc_step,0.8125
epoch,1
trainer/global_step,798
val_loss,0.57918
val_acc,0.79344
train_loss_epoch,0.59331
train_acc_epoch,0.78996


_runtime,▁▁▂▂▂▃▃▃▄▄▄▄▅▅▅▆▆▆▇▇▇▇█
_timestamp,▁▁▂▂▂▃▃▃▄▄▄▄▅▅▅▆▆▆▇▇▇▇█
_step,▁▁▂▂▂▃▃▃▄▄▄▅▅▅▅▆▆▆▇▇▇██
train_loss_step,█▆▇▆▄▆▄▃▃▃▃▁▃▁▂
train_acc_step,▂▅▁▃▄▄▅▇▆▆▅█▅▇▇
epoch,▁▁▁▁▁▁▁▁▁███████████
trainer/global_step,▁▁▂▂▃▃▄▄▄▄▄▅▅▆▆▇▇█████
val_loss,█▁
val_acc,▁█
train_loss_epoch,█▁
train_acc_epoch,▁█


In [23]:
run = wandb.init(project='mlops-project', job_type='producer')

artifact = wandb.Artifact('model', type='model')
artifact.add_dir(MODEL_CKPT_PATH)

run.log_artifact(artifact)
run.join()

wandb: Adding directory to artifact (./model)... Done. 0.0s


In [28]:
model_ckpts = os.listdir(MODEL_CKPT_PATH)
losses = []
for model_ckpt in model_ckpts:
    loss = re.findall("\d+\.\d+", model_ckpt)
    losses.append(float(loss[0]))

if losses:
    losses = np.array(losses)
    best_model_index = np.argsort(losses)[0]
    best_model = model_ckpts[best_model_index]
    print(best_model)
else:
    best_model = None

In [56]:
inference_model = ResNet18.load_from_checkpoint(MODEL_CKPT_PATH+best_model) if best_model else model
y_true, y_pred = evaluate(inference_model, dm.test_dataloader())

In [58]:
from sklearn.preprocessing import label_binarize
from sklearn.metrics import precision_recall_curve

binary_ground_truth = label_binarize(y_true,
                                     classes=np.arange(0, 10).tolist())

precision_micro, recall_micro, _ = precision_recall_curve(binary_ground_truth.ravel(),
                                                          y_pred.ravel())

In [59]:
run = wandb.init(project='mlops-proj', job_type='evaluate')

data = [[x, y] for (x, y) in zip(recall_micro, precision_micro)]
sample_rate = int(len(data)/10000)

table = wandb.Table(columns=["recall_micro", "precision_micro"], data=data[::sample_rate])
wandb.log({"precision_recall" : wandb.plot.line(table, 
                                                "recall_micro", 
                                                "precision_micro", 
                                                stroke=None, 
                                                title="Average Precision")})

run.join()

_runtime,3
_timestamp,1630428437
_step,0


_runtime,▁
_timestamp,▁
_step,▁
